In [193]:
import pandas as pd  # Import pandas and use shorthand notation pd to save on typing
from pandas import Series, DataFrame 
import numpy as np                            
import statsmodels.api as sm 
import scipy as sp
from IPython.core.interactiveshell import InteractiveShell
#InteractiveShell.ast_node_interactivity = "all"

In [194]:
electricityPrices = pd.read_csv(r"C:\Users\Kiran\Documents\University\Economics Yr 3\My Courses\EC3310 Coding\GithubRepo\electricityprices.csv")
electricityPrices['Year'] = electricityPrices['Month beginning'].str[-4:]
del electricityPrices['Month beginning']
electricityPrices = DataFrame({'Average Electricity Price': electricityPrices.groupby('Year')['Price'].mean().round(3)})
electricityPrices.index = electricityPrices.index.astype(int)
electricityPrices

,Average Electricity Price
Year,
2010,44.959
2011,48.203
2012,45.210
2013,50.796
2014,42.432
2015,40.745
2016,42.628
2017,46.312
2018,58.234


In [195]:
busFares = pd.read_excel(r"C:\Users\Kiran\Documents\University\Economics Yr 3\My Courses\EC3310 Coding\GithubRepo\busfares.xlsx")
busFares = busFares.drop(range(13))
busFares = busFares[['Year','Great Britain']]
busFares = busFares.set_index('Year')
busFares.columns = ['Bus Fare Cost']
busFares

,Bus Fare Cost
Year,
2008,113.356652
2009,123.078460
2010,128.952627
2011,135.192095
2012,143.355417
2013,150.127342
2014,154.740813
2015,159.710623
2016,162.633562


In [196]:
total = pd.concat([electricityPrices, busFares], axis=1)
total

,Average Electricity Price,Bus Fare Cost
Year,,
2008,NaN,113.356652
2009,NaN,123.078460
2010,44.959,128.952627
2011,48.203,135.192095
2012,45.210,143.355417
2013,50.796,150.127342
2014,42.432,154.740813
2015,40.745,159.710623
2016,42.628,162.633562


In [197]:
charger_data = {'Slow':[1325, 1611, 2126, 2448, 2228, 2451, 2743, 3293, 5947], 
        'Fast': [178,1193,3309,4734,5946,7143,8754,11459,16457], 
        'Rapid':[34,79,284,844,1710,2153,2663,4025,6212],
        'Ultra-rapid':[None,None,None,None,None,None,None,341,876]}
df = pd.DataFrame(charger_data, index=[2011,2012,2013,2014,2015,2016,2017,2018,2019])
df["Sum"] = df.sum(axis=1).astype(int)
df

,Slow,Fast,Rapid,Ultra-rapid,Sum
2011,1325,178,34,NaN,1537
2012,1611,1193,79,NaN,2883
2013,2126,3309,284,NaN,5719
2014,2448,4734,844,NaN,8026
2015,2228,5946,1710,NaN,9884
2016,2451,7143,2153,NaN,11747
2017,2743,8754,2663,NaN,14160
2018,3293,11459,4025,341.0,19118
2019,5947,16457,6212,876.0,29492


In [198]:
total['Charging Connectors'] = df['Sum']
total['ULEZ Variable'] = 1
range_data = [None, 244, 84, 73, 118, 79, 81, 114, 128, 177, 190, 220, 256]
total['Range of Electric Vehicle (miles)'] = range_data

In [199]:
electricityAvailable = pd.read_excel(r"C:\Users\Kiran\Documents\University\Economics Yr 3\My Courses\EC3310 Coding\GithubRepo\electricityavailable.xlsx")
electricityAvailable = electricityAvailable.set_index("Year") 
total = pd.concat([total, electricityAvailable], axis=1)
total = total.rename({'Electricity Available': 'Electricity Available (TWh)'}, axis=1).round(3)
total

,Average Electricity Price,Bus Fare Cost,Charging Connectors,ULEZ Variable,Range of Electric Vehicle (miles),Electricity Available (TWh)
Year,,,,,,
2008,NaN,113.357,NaN,1,NaN,378.202
2009,NaN,123.078,NaN,1,244.0,358.167
2010,44.959,128.953,NaN,1,84.0,364.397
2011,48.203,135.192,1537.0,1,73.0,353.909
2012,45.210,143.355,2883.0,1,118.0,353.776
2013,50.796,150.127,5719.0,1,79.0,350.935
2014,42.432,154.741,8026.0,1,81.0,338.253
2015,40.745,159.711,9884.0,1,114.0,339.618
2016,42.628,162.634,11747.0,1,128.0,337.653


In [200]:
import requests
import json

api = requests.get('https://api.ons.gov.uk/dataset/MM23/timeseries/CJXN/data')
 
# print (api.json())
newcars_dict = json.loads(api.content)
years = newcars_dict['years']
filtered = []
for year in years:
    if int(year['year']) > 2007:
        filtered.append(int(float(year['value'])))
newCars = np.asarray(filtered)
total['Price of New Cars'] = newCars
total

,Average Electricity Price,Bus Fare Cost,Charging Connectors,ULEZ Variable,Range of Electric Vehicle (miles),Electricity Available (TWh),Price of New Cars
Year,,,,,,,
2008,NaN,113.357,NaN,1,NaN,378.202,26
2009,NaN,123.078,NaN,1,244.0,358.167,26
2010,44.959,128.953,NaN,1,84.0,364.397,23
2011,48.203,135.192,1537.0,1,73.0,353.909,25
2012,45.210,143.355,2883.0,1,118.0,353.776,24
2013,50.796,150.127,5719.0,1,79.0,350.935,21
2014,42.432,154.741,8026.0,1,81.0,338.253,23
2015,40.745,159.711,9884.0,1,114.0,339.618,24
2016,42.628,162.634,11747.0,1,128.0,337.653,25


In [201]:
from bs4 import BeautifulSoup
import re

url = ('https://www.ons.gov.uk/economy/grossdomesticproductgdp/timeseries/k8a5/bb')
page = requests.get(url).text
soup = BeautifulSoup(page, "html.parser")
rows = soup.find_all("table")[1].find('tbody').find_all('tr')

co_data = []
for row in rows:
    cells = row.find_all("td")
    years = int(cells[0].text)
    data = int(float(cells[1].text))
    co_data.append([years, data])

co_df = pd.DataFrame(co_data, columns=['Year', 'Carbon Monoxide (thousand tonnes)'])
co_df = co_df.drop(range(18))
co_df = co_df.set_index('Year')
total = pd.concat([total, co_df], axis=1)
total

,Average Electricity Price,Bus Fare Cost,Charging Connectors,ULEZ Variable,Range of Electric Vehicle (miles),Electricity Available (TWh),Price of New Cars,Carbon Monoxide (thousand tonnes)
Year,,,,,,,,
2008,NaN,113.357,NaN,1,NaN,378.202,26,2555.0
2009,NaN,123.078,NaN,1,244.0,358.167,26,2132.0
2010,44.959,128.953,NaN,1,84.0,364.397,23,2052.0
2011,48.203,135.192,1537.0,1,73.0,353.909,25,1872.0
2012,45.210,143.355,2883.0,1,118.0,353.776,24,1849.0
2013,50.796,150.127,5719.0,1,79.0,350.935,21,1847.0
2014,42.432,154.741,8026.0,1,81.0,338.253,23,1765.0
2015,40.745,159.711,9884.0,1,114.0,339.618,24,1722.0
2016,42.628,162.634,11747.0,1,128.0,337.653,25,1593.0
